In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [2]:
urls = ['https://www.amazon.com/Chefman-Liter-Electric-Glass-Kettle/dp/B07KYYF9HD/ref=sr_1_1?dchild=1&keywords=816458023023&qid=1586630945&sr=8-1', 'https://www.amazon.com/Instant-Vortex-One-Touch-Programs-Reheat/dp/B07VM1FT8W/ref=sr_1_1?dchild=1&keywords=857561008637&qid=1586627447&s=home-garden&sr=1-1', 'https://www.amazon.com/CeraVe-Foaming-Facial-Cleanser-Washing/dp/B01N1LL62W/ref=sr_1_5?dchild=1&keywords=cerave&qid=1586628117&s=home-garden&sr=8-5']

questions:
    How would items in 2 packs work?

In [3]:
dict_test = {}

In [ ]:
'''This is persistent dictionaries: write each dictionary as a file to local computer
* Might not need this at all if final project uses jupyter notebook'''


# ASIN_dict
# key: ASIN, value: UPC
try:
	with open('asin_dict.pickle', 'rb') as file:
		ASIN_dict = pickle.load(file)
except FileNotFoundError:
	ASIN_dict = dict()

# UPC_dict
# key: UPC, value: product title
try:
	with open('upc_dict.picle', 'rb') as file:
		UPC_dict = pickle.load(file)
except FileNotFoundError:
	UPC_dict = dict()

In [4]:
amazon_name = []
amazon_price = []
amazon_time = []
target_name = []
target_price = [] 
walmart_name = []
walmart_price = []

In [5]:
def scraper(urls):
    
    for url in urls:
        
        driver = Chrome(executable_path=r'C:/Users/Tinatin/Scraping/chromedriver.exe')
        driver.get(url)
        amazon_name.append(driver.find_element_by_xpath('//*[@id="productTitle"]').text)
        amazon_price.append(driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text)
        amazon_time.append(str(datetime.today()))
        asin = url.split('/')[5]
        driver.quit()
        if asin in dict_test:
            upc_id = dict_test[asin]
        # Getting UPC from ASIN
        else:
            upc_driver = Chrome(executable_path=r'C:/Users/Tinatin/Scraping/chromedriver.exe')
            upc_driver.get('https://www.synccentric.com/features/upc-asin/')
            box = upc_driver.find_element_by_id('scrollto')
            form = box.find_element_by_class_name('form-group')
            input = form.find_element_by_name('identifier')
            input.send_keys(asin)
            input.submit()
            time.sleep(10)
            id_list = upc_driver.find_element_by_class_name('col-sm-8').find_elements_by_tag_name('strong')
            upc_id = id_list[1].get_attribute('innerHTML')
            upc_driver.quit()

            # Target UPC 
        target_driver = Chrome(executable_path=r'C:/Users/Tinatin/Scraping/chromedriver.exe')
        target_url = 'https://www.target.com/s?searchTerm=' + upc_id
        target_driver.get(target_url)
        time.sleep(10)
        #target_driver.find_element_by_xpath("//a[contains(@href, '{}')]".format(name.split(' ')[1].lower())).click()
        #name = target_driver.find_element_by_xpath('//*[@id="viewport"]/div[4]/div/div[1]/div[2]/h1/span').text
        #price = target_driver.find_element_by_xpath('//*[@id="viewport"]/div[4]/div/div[2]/div[2]/div[1]/div[1]/div[1]').text
        name = target_driver.find_element_by_xpath('//*[@id="mainContainer"]/div[3]/div[2]/div/div[1]/div[3]/div/ul/li/div/div[2]/div/div/div/div[1]/div[1]/a').text
        
        if len(name) == 0:
            target_name.append("no name found")
        else:
            target_name.append(name)
        price = target_driver.find_element_by_xpath('//*[@id="mainContainer"]/div[3]/div[2]/div/div[1]/div[3]/div/ul/li/div/div[2]/div/div/div/div[2]/span').text
        if len(price) == 0:
            target_price.append("no price found")
        else:
            target_price.append(price)
            
        target_driver.quit()
        walmart_driver = Chrome(executable_path=r'C:/Users/Tinatin/Scraping/chromedriver.exe')
        walmart_url = 'https://www.walmart.com/search/?query=' + upc_id
        walmart_driver.get(walmart_url)
        
        names_prices = walmart_driver.find_elements_by_xpath("//div[contains(@class, 'tile-content Grid-col u-size-8-10-l list-description-wrapper')]")
        if len(names_prices) == 0:
            walmart_name.append('no name found')
            walmart_price.append('no price found')
        else:
            for i in names_prices:
                if 'Pack' not in i.text:
                    walmart_name.append(i.text.split('\n')[1])
                    walmart_price.append(i.text.split('\n')[9])
            
                
        walmart_driver.quit()

In [6]:
# while True:
#     scraper(urls)
#     time.sleep(10)

In [7]:
i = 1
while i <3:
    print(i)
    scraper(urls)
    time.sleep(10)
    i += 1

1
2


In [8]:
prices = pd.DataFrame({'amazon_name' : amazon_name,
          'amazon_price' : amazon_price,
          'amazon_time' : amazon_time,
            'target_name': target_name,
            'target_price': target_price,
             'walmart_name': walmart_name,
             'walmart_price': walmart_price}, 
         columns=['amazon_name','amazon_price', 'amazon_time','target_name','target_price','walmart_name', 'walmart_price' ])    

In [9]:
prices

,amazon_name,amazon_price,amazon_time,target_name,target_price,walmart_name,walmart_price
0,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$19.99,2020-04-11 21:02:10.310191,Chefman 1.7L Glass Electric Kettle - Clear,$19.99,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$24.99
1,"Instant Vortex 4-in-1 Air Fryer, 6 Quart, 4 On...",$99.00,2020-04-11 21:03:20.662978,Instant Pot 6qt Vortex Air Fryer,$99.99,no name found,no price found
2,CeraVe Foaming Facial Cleanser | 16 Fl Oz | Da...,$14.13,2020-04-11 21:04:34.692557,CeraVe Foaming Facial Cleanser for Normal to O...,$14.99,Cerave Oil Control Value Size Foaming Facial C...,$15.64
3,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$19.99,2020-04-11 21:05:55.926576,Chefman 1.7L Glass Electric Kettle - Clear,$19.99,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$24.99
4,"Instant Vortex 4-in-1 Air Fryer, 6 Quart, 4 On...",$99.00,2020-04-11 21:07:20.548234,Instant Pot 6qt Vortex Air Fryer,$99.99,no name found,no price found
5,CeraVe Foaming Facial Cleanser | 16 Fl Oz | Da...,$14.13,2020-04-11 21:08:33.725646,CeraVe Foaming Facial Cleanser for Normal to O...,$14.99,Cerave Oil Control Value Size Foaming Facial C...,$15.64


In [3]:
# MAIN

buyer_type = input('Are you an Informed Buyer (I) or a Window Shopper (W)? ').strip().upper()

# This whole if can be converted to a function to add more URLs to the urls list

if buyer_type == 'I':
	basket = []
	item = input('Paste Amazon link of the product you want to track or press Enter key when done ')
	while True:
		if item == '':
			break
		else:		
			basket.append(item)
			item = input('Paste Amazon link of the product you want to track or press Enter key when done ')
	# print('Your basket: ', basket)	

# 	informed_buyer(basket)


else:    # window shopper
	pass

Are you an Informed Buyer (I) or a Window Shopper (W)? I
Paste Amazon link of the product you want to track or press Enter key when done 
